In [1]:
import scanpy as sc
from pathlib import Path
import tarfile
import pySingleCellNet as cn
import anndata as ad
import sys
sys.path.append('../src')

#from preprocessing import annotation_preprocess
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import cna_tool
from cna_tool import CNAInferer  # assuming this contains your working class
from cna_tool.utils import select_control_mask, map_gene_coordinates
from cna_tool.infer import infer_cnas_from_scrna
from cna_tool.tl import run_cna_evaluation
from cna_tool.cna_inference import test_pipeline_on_slice
from mygene import MyGeneInfo
from cna_tool.preprocessing import simple_preprocess

c:\Users\clark\anaconda3\envs\OneSC_run\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
adata_test = ad.read_h5ad('../data/test_adata.h5ad')
adata7 = ad.read_h5ad('../data/num7_adata.h5ad')
adata4 = ad.read_h5ad('../data/num4_adata.h5ad')
adata3 = ad.read_h5ad('../data/num3_adata.h5ad')
adata6 = ad.read_h5ad('../data/num6_adata.h5ad')


c:\Users\clark\anaconda3\envs\OneSC_run\lib\site-packages\anndata\_core\anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [49]:
adata7 = cna_tool.preprocessing.simple_preprocess(adata7)
adata7.var["gene_name"] = adata7.var_names
adata7.obs["cell_type"] = "unknown"
adata7 = cna_tool.utils.map_gene_coordinates(adata7, adata_test)
adata7_filtered = adata7[:, ~adata7.var['chromosome'].isna()].copy()
adata7_filtered.write_h5ad('../data/preprocessed_data/num7_adata_filtered.h5ad')


Numer of genes in dataset: 36601
Number of common genes: 19501


In [50]:
adata6 = cna_tool.preprocessing.simple_preprocess(adata6)
adata6.var["gene_name"] = adata6.var_names
adata6.obs["cell_type"] = "unknown"
adata6 = cna_tool.utils.map_gene_coordinates(adata6, adata_test)
adata6_filtered = adata6[:, ~adata6.var['chromosome'].isna()].copy()
adata6_filtered.write_h5ad('../data/preprocessed_data/num6_adata_filtered.h5ad')

Numer of genes in dataset: 45068
Number of common genes: 20104


In [51]:
adata3 = cna_tool.preprocessing.simple_preprocess(adata3)
adata3.var["gene_name"] = adata3.var_names
adata3.obs["cell_type"] = "unknown"
adata3 = cna_tool.utils.map_gene_coordinates(adata3, adata_test)
adata3_filtered = adata3[:, ~adata3.var['chromosome'].isna()].copy()
adata3_filtered.write_h5ad('../data/preprocessed_data/num3_adata_filtered.h5ad')

c:\Users\clark\anaconda3\envs\OneSC_run\lib\site-packages\scanpy\preprocessing\_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


Numer of genes in dataset: 37494


c:\Users\clark\anaconda3\envs\OneSC_run\lib\site-packages\anndata\_core\anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Number of common genes: 19302


c:\Users\clark\anaconda3\envs\OneSC_run\lib\site-packages\anndata\_core\anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


# Dataset 4 needs to process differently

In [53]:
adata4 = ad.read_h5ad('../data/num4_adata.h5ad')
adata4 = cna_tool.preprocessing.simple_preprocess(adata4)
adata4.var["gene_name"] = adata4.var_names
adata4.obs["cell_type"] = "unknown"

# Convert to string to avoid errors
adata4.var['gene_name'] = adata4.var['gene_name'].astype(str)

# Masks for human and mouse genes
human_mask = adata4.var['gene_name'].str.startswith('GRCh38_')
mouse_mask = adata4.var['gene_name'].str.startswith('mm10')

# Create separate AnnData objects
adata4_human = adata4[:, human_mask].copy()
adata4_mouse = adata4[:, mouse_mask].copy()
adata4_human.var['gene_name'] = adata4_human.var['gene_name'].str.replace('^GRCh38_', '', regex=True)
adata4_human.var_names = adata4_human.var["gene_name"]
adata4_mouse.var['gene_name'] = adata4_mouse.var['gene_name'].str.replace('^mm10___', '', regex=True)
adata4_mouse.var_names = adata4_mouse.var["gene_name"]

adata4_human = cna_tool.utils.map_gene_coordinates(adata4_human, adata_test)
adata4_mouse = cna_tool.utils.map_gene_coordinates(adata4_mouse, adata_test)

# write
adata4_human_filtered = adata4_human[:, ~adata4_human.var['chromosome'].isna()].copy()
adata4_human_filtered.write_h5ad('../data/preprocessed_data/num4_human_adata_filtered.h5ad')
adata4_mouse_filtered = adata4_mouse[:, ~adata4_mouse.var['chromosome'].isna()].copy()
adata4_mouse_filtered.write_h5ad('../data/preprocessed_data/num4_mouse_adata_filtered.h5ad')

Numer of genes in dataset: 36601
Number of common genes: 19495
Numer of genes in dataset: 32285
Number of common genes: 9


In [54]:
adata4_human.var

,gene_name,chromosome,start,end
gene_name,,,,
MIR1302-2HG,MIR1302-2HG,NaN,NaN,NaN
FAM138A,FAM138A,NaN,NaN,NaN
OR4F5,OR4F5,NaN,NaN,NaN
AL627309.1,AL627309.1,NaN,NaN,NaN
AL627309.3,AL627309.3,1,89551.0,91105.0
...,...,...,...,...
AC141272.1,AC141272.1,NaN,NaN,NaN
AC023491.2,AC023491.2,NaN,NaN,NaN
AC007325.1,AC007325.1,NaN,NaN,NaN


In [ ]:

def load_gtf_as_dataframe(gtf_file):
    rows = []
    with open(gtf_file, 'r') as f:
        for line in f:
            if line.startswith("#"):
                continue
            fields = line.strip().split('\t')
            if fields[2] != 'gene':
                continue
            chrom, source, feature, start, end, score, strand, frame, attr = fields
            info = {k: v.strip('"') for k, v in 
                    [field.strip().split(' ')[:2] for field in attr.strip(';').split(';') if field]}
            rows.append({
                'gene_name': info.get('gene_name'),
                'gene_id': info.get('gene_id'),
                'chromosome': chrom.replace('chr', ''),
                'start': int(start),
                'end': int(end)
            })
    return pd.DataFrame(rows)

gtf_df = load_gtf_as_dataframe("gencode.v47.chr_patch_hapl_scaff.annotation.gtf")

In [33]:
def fetch_gene_coordinates_via_mygene(adata, species='human'):
    mg = MyGeneInfo()
    gene_names = adata.var['gene_name'].astype(str).unique().tolist()

    print(f"Querying {len(gene_names)} genes from mygene.info...")
    results = mg.querymany(gene_names, scopes='symbol', fields='genomic_pos', species=species, as_dataframe=True)
    results = results[~results.index.duplicated(keep='first')]  # remove duplicate matches

    def get_coord(gene, field):
        try:
            val = results.loc[gene, 'genomic_pos']
            if isinstance(val, dict):
                return val.get(field)
            elif isinstance(val, list) and isinstance(val[0], dict):
                return val[0].get(field)
            else:
                return None
        except Exception:
            return None

    adata.var['chromosome'] = adata.var['gene_name'].map(lambda g: get_coord(g, 'chr'))
    adata.var['start'] = adata.var['gene_name'].map(lambda g: get_coord(g, 'start'))
    adata.var['end'] = adata.var['gene_name'].map(lambda g: get_coord(g, 'end'))

    return adata


In [44]:
def gene_info(gtf_df, adata):

    #adata.var['gene_name'] = adata.var.index

    # Fetch gene coordinates using mygene
    adata = fetch_gene_coordinates_via_mygene(adata)
    print(adata.var[['gene_name', 'chromosome', 'start', 'end']].head(10))
    print("Missing values:", adata.var[['chromosome', 'start', 'end']].isna().sum())
    print("done fetching ...")

    # merge gene coordinates with GTF
    # Step 1: Copy and reset gene_name
    merged = adata.var.copy()
    merged['gene_name'] = merged.index.astype(str)

    # Step 2: Merge with GTF
    merged = merged.merge(gtf_df, on='gene_name', how='left')

    # Step 3: Drop duplicates (if merge resulted in more rows)
    merged = merged.drop_duplicates(subset='gene_name', keep='first')

    # Step 4: Align merged index with adata.var
    merged = merged.set_index('gene_name')
    merged = merged.loc[adata.var.index]  # ensure same order and rows

    # Step 5: Assign to adata.var
    adata.var['chromosome'] = merged['chromosome_y'].fillna(merged['chromosome_x'])
    adata.var['start'] = merged['start_y'].fillna(merged['start_x'])
    adata.var['end'] = merged['end_y'].fillna(merged['end_x'])

    print(adata.var[['chromosome', 'start', 'end']].isna().sum())
    return adata

In [46]:
adata4_mouse = gene_info(gtf_df, adata4_mouse)

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


Querying 32285 genes from mygene.info...


104 input query terms found dup hits:	[('Snhg6', 2), ('Dnajb3', 2), ('Dnm3os', 2), ('Mptx1', 3), ('Nron', 2), ('Tmco5b', 2), ('Mir124-2hg'
16249 input query terms found no hit:	['Gm1992', 'Gm19938', 'Gm37381', 'Gm37587', 'Gm37323', 'Gm16041', '4732440D04Rik', 'Gm26901', 'Gm304


               gene_name chromosome  start  end
mm10___Xkr4         Xkr4       None    NaN  NaN
mm10___Gm1992     Gm1992       None    NaN  NaN
mm10___Gm19938   Gm19938       None    NaN  NaN
mm10___Gm37381   Gm37381       None    NaN  NaN
mm10___Rp1           Rp1       None    NaN  NaN
mm10___Sox17       Sox17       None    NaN  NaN
mm10___Gm37587   Gm37587       None    NaN  NaN
mm10___Gm37323   Gm37323       None    NaN  NaN
mm10___Mrpl15     Mrpl15       None    NaN  NaN
mm10___Lypla1     Lypla1       None    NaN  NaN
Missing values: chromosome    31258
start         31258
end           31258
dtype: int64
done fetching ...
chromosome    31258
start         31258
end           31258
dtype: int64


In [32]:
adata4_mouse.var

,gene_name,chromosome,start,end
mm10___Xkr4,mm10___Xkr4,NaN,NaN,NaN
mm10___Gm1992,mm10___Gm1992,NaN,NaN,NaN
mm10___Gm19938,mm10___Gm19938,NaN,NaN,NaN
mm10___Gm37381,mm10___Gm37381,NaN,NaN,NaN
mm10___Rp1,mm10___Rp1,NaN,NaN,NaN
...,...,...,...,...
mm10___AC124606.1,mm10___AC124606.1,NaN,NaN,NaN
mm10___AC133095.2,mm10___AC133095.2,NaN,NaN,NaN
mm10___AC133095.1,mm10___AC133095.1,NaN,NaN,NaN
mm10___AC234645.1,mm10___AC234645.1,NaN,NaN,NaN
